In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install transformers

     |████████████████████████████████| 3.4 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 5.7 MB/s 
     |████████████████████████████████| 596 kB 54.2 MB/s 
     |████████████████████████████████| 61 kB 689 kB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from transformers import BertTokenizer, BertModel, DistilBertConfig, TFDistilBertModel, DistilBertModel, DistilBertTokenizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [4]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

## Preprocess 

### Clothing Review Dataset

In [5]:
# # Load dataset for clothing reviews
# reviews = pd.read_csv("/content/drive/MyDrive/data /Womens Clothing E-Commerce Reviews.csv")
# reviews = reviews.dropna()
# print(reviews.shape)
# reviews.head()
# Load dataset for clothing reviews


spam = pd.read_csv("/content/drive/MyDrive/data /spam.csv",encoding="latin1")
spam.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
spam = spam[['v2', 'v1']]
spam.columns = ['content', 'detection']
spam['content_length'] = spam['content'].apply(lambda x: len(x.split()))
spam.head()

,content,detection,content_length
0,"Go until jurong point, crazy.. Available only ...",ham,20
1,Ok lar... Joking wif u oni...,ham,6
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,28
3,U dun say so early hor... U c already then say...,ham,11
4,"Nah I don't think he goes to usf, he lives aro...",ham,13


In [6]:
# reviews['Title'] = reviews['Title'].fillna('')
# reviews['Review Text'] = reviews['Review Text'].fillna('')
# reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']
tracker = []
for aDetection in spam['detection']: 
  if not(aDetection in tracker): 
    tracker.append(aDetection)
print(len(tracker))

number_Detection = {}
index = 0
for aDetection in tracker: 
  number_Detection[aDetection] = index 
  index = index + 1
spam['detection'] = spam['detection'].apply(lambda x: number_Detection[x])

2


In [7]:
# reviews = reviews[['review', 'Rating']]
# reviews.columns = ['review', 'rating']
# reviews.head()
spam = spam[['content', 'detection']]
spam.columns = ['content', 'detection']
spam.head()

,content,detection
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


#### Tokenize each sentence

In [8]:
#take advantage of nltk to tokenize all sentences
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
en_stop_words = set(stopwords.words('english'))
tokenizer = nltk.RegexpTokenizer(r'\w+|\$[\d\.]+')

def tokenize_sent(sent):
    
    tokenized = tokenizer.tokenize(sent)
    filtered = [w.lower() for w in tokenized if w.lower() not in en_stop_words]
    return filtered
spam['tokenized'] = spam['content'].apply(lambda x: tokenize_sent(x))
# reviews['tokenized'] = reviews['review'].apply(lambda x: tokenize_sent(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# Sanity cehck for tokenizers
print(spam.head(3))

                                             content  ...                                          tokenized
0  Go until jurong point, crazy.. Available only ...  ...  [go, jurong, point, crazy, available, bugis, n...
1                      Ok lar... Joking wif u oni...  ...                     [ok, lar, joking, wif, u, oni]
2  Free entry in 2 a wkly comp to win FA Cup fina...  ...  [free, entry, 2, wkly, comp, win, fa, cup, fin...

[3 rows x 3 columns]


#### Encode and truncate sentence

In [10]:
# # sanity check for encoding:
# print(len(all_words))
# print(ave_len)
# lengths = [len(x) for x in reviews.encoded]
# print(max(lengths))
# print(min(lengths))
# print(set(reviews.rating))
# review_length = reviews.review_length
# reviews.head()
# print(len(word_set))

In [11]:
# get unique words in the corpus
all_words = []
for x in spam['tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in spam.tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in spam['tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in reviews.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

spam['encoded'] = spam.tokenized.apply(lambda x: encode(x))

print(spam.head())

# get sequence average length
total_len = 0
for x in spam.encoded:
    total_len += len(x)

ave_len = math.floor(total_len/spam.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

for i, row in spam.iterrows():
    size = min(len(row.encoded), ave_len)
    new_encoded = row.encoded[:size]
    if size < ave_len:
        for j in range(0, ave_len - len(row.encoded)):
            new_encoded.append(1)
    spam.at[i, 'encoded'] = new_encoded

spam['review_length'] = spam.encoded.apply(lambda x: len(x))


                                             content  ...                                            encoded
0  Go until jurong point, crazy.. Available only ...  ...  [3994, 6432, 3673, 4134, 2503, 3303, 5730, 93,...
1                      Ok lar... Joking wif u oni...  ...                [3448, 5979, 7405, 708, 4183, 2668]
2  Free entry in 2 a wkly comp to win FA Cup fina...  ...  [5916, 3174, 323, 6898, 4494, 5094, 8093, 5394...
3  U dun say so early hor... U c already then say...  ...  [4183, 6827, 8449, 4495, 6120, 4183, 755, 5697...
4  Nah I don't think he goes to usf, he lives aro...  ...          [5953, 1794, 5366, 4638, 6186, 4729, 958]

[5 rows x 4 columns]


In [12]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in spam.encoded]
print(max(lengths))
print(min(lengths))
print(set(spam.detection))
review_length = spam.review_length
spam.head()
print(len(word_set))

53413
9
9
9
{0, 1}
8577


In [13]:
# # Train test split from skearln
# data_size = len(reviews['encoded'])
# assert data_size == len(reviews['rating']) 
# X, y = list(zip(list(reviews['encoded']),(list(reviews['review_length'])))), list(reviews['rating'])
# # X(data, length)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)
# Train test split from skearln
data_size = len(spam['encoded'])
assert data_size == len(spam['detection']) 
X, y = list(zip(list(spam['encoded']),(list(spam['review_length'])))), list(spam['detection'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [14]:
# Sanity check for length match
print(X_train[0])

([7001, 1378, 7078, 5411, 5147, 1378, 7090, 1410, 4866], 9)


## Dataset and Dataloader

### Clothing Review Dataset

In [15]:
class Dataset_word2vec(Dataset):
    def __init__(self, X, y): 
        self.X = X

        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [16]:
trainSet_w2v = Dataset_word2vec(X_train, y_train)
testSet_w2v = Dataset_word2vec(X_test, y_test)

In [17]:
print(trainSet_w2v[0])

(([7001, 1378, 7078, 5411, 5147, 1378, 7090, 1410, 4866], 9), 0)


In [18]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [19]:
trainLoader_w2v = DataLoader(dataset = trainSet_w2v, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_w2v = DataLoader(dataset = testSet_w2v, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)

In [20]:
it = iter(trainLoader_w2v)
x, y, seq_len = it.next()

In [21]:
x.data

tensor([[5916, 3174, 7553, 6898, 4494, 8172, 3903, 5094, 7532],
        [2625, 1816, 7306, 6826, 5375, 2890, 4229, 3855, 2421],
        [6862, 7045, 2103, 6363,    1,    1,    1,    1,    1],
        [4688, 3239, 6535, 7135, 1062, 2196,    1,    1,    1],
        [2698, 3994, 4567, 3152, 7873, 7328,  908, 5531,    1],
        [4294, 6123, 6088, 8285, 1816, 2976, 7580, 2371, 4183],
        [2421, 4871, 5708, 2621, 2669, 2421, 6427,  567, 6918],
        [3827,  883, 7717, 2931, 7482, 8315, 5111, 6969, 7510],
        [8239, 2667, 4295, 1960,    1,    1,    1,    1,    1],
        [3871, 8216, 8316, 1159, 4621, 6572, 2088, 3599, 6437],
        [7909, 2552,  927, 2299, 5827, 6476, 6313,  456, 2799],
        [7280, 7078, 6577, 5799, 7078, 5137, 1757, 7078, 6403],
        [3684, 5076, 8285, 4703, 8285, 2153, 4183, 8215, 7602],
        [2698, 6055, 6427, 3318, 2698, 4267,    1,    1,    1],
        [6226, 2583, 5884, 5711, 1742, 4176,    1,    1,    1],
        [3077,  445, 8213,  883, 3544, 5

In [22]:
y

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])

In [23]:
seq_len

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

## Embedding

In [24]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/data /GoogleNews-vectors-negative300.bin', binary = True)
print(type(word2vec))
weight = torch.FloatTensor(word2vec.vectors)
word2vec_embedding = nn.Embedding.from_pretrained(weight)

# sanity check for word2vec embedding
temp = word2vec['hello', 'world']
sanity_w2v = torch.Tensor(temp)
print(sanity_w2v.shape)

<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>
torch.Size([2, 300])


In [25]:
# create embedding matrix lstm
def emb_matrix(model, all_words, word2index):
    matrix_size = len(word_set) + 2
    emb_matrix = np.zeros((matrix_size, 300))
    emb_matrix[0] = np.random.uniform(-0.25, 0.25, 300) # vector for UNK
    emb_matrix[1] = np.zeros(300) # vector for padding, has no weight

    # vector for every other word in the dictionary
    for i in range(2, len(all_words)+2):
        word = all_words[i-2]
        index = word2index[word]
        try:
            vector = model[word]
        except Exception as e:
            # word does not exist in the pretrained embedding
            vector = emb_matrix[0]
        emb_matrix[index] = vector
    return emb_matrix

In [26]:
embeddings_w2v = emb_matrix(word2vec, word_set, word2index)

In [27]:
# print(embedding.shape)

In [28]:
# print(embedding)

In [29]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model
    y_total = []
    y_pred_total = []
    for x, y, seq_len in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        y_total.extend(y.tolist())
        y_pred_total.extend(pred.tolist())
        
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        # print(len(y_total))
        # print(len(y_pred_total))

      # calculate precision, recall, and f1
    f1 = (f1_score(y_total, y_pred_total, average='weighted'))
    precision = (precision_score(y_total, y_pred_total, average='weighted'))
    recall =(recall_score(y_total, y_pred_total, average='weighted'))
    return sum_loss/total, correct/total, precision, recall, f1

In [30]:
def train_model(model, fname, epochs = 50, lr = 0.0001):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model
    PATH = '/content/drive/MyDrive/McGill/Comp 550/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in range(epochs):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in tqdm(trainLoader_w2v):
            x = x.long()
            y = y.long()
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, precision, recall, f1 = validation_metrics(model, testLoader_w2v)
        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f, precision %.3f, recall %.3f, F1 %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc, precision, recall, f1))
        if val_acc > best_val_acc and i>=1:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}.pth'
            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [31]:
weight = torch.FloatTensor(word2vec.vectors)
word2vec_embedding = nn.Embedding.from_pretrained(weight)

class LSTM_word2vec(torch.nn.Module):
    def __init__(self, emb_matrix, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(emb_matrix.shape[0], embedding_dim, padding_idx = 0)
        self.embeddings.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, seq_len):
        x = self.embeddings(x)
        # x = nn.utils.rnn.pack_padded_sequence(x, self.embedding_dim, batch_first = True)
        x = self.dropout(x)
        # x_padded = pack_padded_sequence(x, seq_len, batch_first = True)
        output, (h_n, c_n) = self.lstm(x)
        # print(h_n[-1].shape)
        # output (N, ave_len, num_class)
        # reshape to out (N, num_class)
        
        return self.linear(h_n[-1])

In [32]:
# distil_bert = 'distilbert-base-uncased'
#         #config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
#         #config.output_hidden_states = False
# transformer_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
# embedding =  list(transformer_model.children())[0]
# bert_word_embeddings = list(embedding.children())[0]

In [33]:
# class LSTM_fixed_length_Bert(torch.nn.Module) :
#     def __init__(self, vocab_size, embedding_dim, hidden_dim) :
#         super().__init__()
#         self.embeddings = bert_word_embeddings
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.linear = nn.Linear(hidden_dim, 5)
#         self.dropout = nn.Dropout(0.5)
        
#     def forward(self, x, l):
#         x = self.embeddings(x)
#         x = self.dropout(x)
#         lstm_out, (ht, ct) = self.lstm(x)
#         return self.linear(ht[-1])

In [34]:
# model_bert = LSTM_fixed_length_Bert(len(word_set), 768, 50)
# train_model(model_bert, fname = 'bert_lstm_clothing', epochs = 30)

In [35]:
model = LSTM_word2vec(embeddings_w2v, 300, 100)
train_model(model, fname='word2vec_lstm', epochs=10)

At epoch 0


100%|██████████| 278/278 [00:05<00:00, 49.78it/s]


current in vlaidaiton
train loss 0.815, train accuracy 0.748, val loss 0.313, val accuracy 0.872, precision 0.879, recall 0.872, F1 0.823
At epoch 1


100%|██████████| 278/278 [00:05<00:00, 53.52it/s]


current in vlaidaiton
train loss 0.185, train accuracy 0.936, val loss 0.139, val accuracy 0.954, precision 0.953, recall 0.954, F1 0.953
	=> Best model saved at 1th epoch with valication accuracy of 0.9538043737411499
At epoch 2


...

current in vlaidaiton
train loss 0.020, train accuracy 0.996, val loss 0.076, val accuracy 0.975, precision 0.974, recall 0.975, F1 0.974
At epoch 8


100%|██████████| 278/278 [00:05<00:00, 53.07it/s]


current in vlaidaiton
train loss 0.018, train accuracy 0.996, val loss 0.071, val accuracy 0.976, precision 0.975, recall 0.976, F1 0.975
	=> Best model saved at 8th epoch with valication accuracy of 0.9755434989929199
At epoch 9


100%|██████████| 278/278 [00:05<00:00, 53.24it/s]


current in vlaidaiton
train loss 0.013, train accuracy 0.998, val loss 0.073, val accuracy 0.976, precision 0.976, recall 0.976, F1 0.976
	=> Best model saved at 9th epoch with valication accuracy of 0.9764492511749268
